In [8]:
import pandas as pd
from dotenv import load_dotenv
load_dotenv(override=True)
df = pd.read_csv('../curated/DEVRT-DACIA-SPRING.csv')

In [16]:
from data_analysis_agent.automotive_data_quality import generate_automotive_quality_report

In [30]:
# Final demonstration: Silent mode when writing to files
print("Running automotive quality report with JSON output...")
generate_automotive_quality_report(df, json_output_file='report.json')
print("Report completed! Check report.json for results.")

INFO:data_analysis_agent.automotive_data_quality:JSON report saved to report.json


Running automotive quality report with JSON output...
Report completed! Check report.json for results.


In [31]:
df_questions = pd.read_json('../curated/15_DEVRT-DACIA-SPRING-questions.jsonl', lines=True)
df_answers = pd.read_json('../curated/15_DEVRT-DACIA-SPRING-labels.jsonl', lines=True)

In [32]:
df_questions['question'].head(), df_answers['common_answers'].head()

(0    What is the correlation coefficient between al...
 1    What is the average altitude I drive at? Fill ...
 2    Which drivers achieve the best energy efficien...
 3    What is the average state of health (SOH) perc...
 4    What is the correlation coefficient between fr...
 Name: question, dtype: object,
 0        {'altitude_speed_correlation': -0.03}
 1           {'average_altitude_meters': 135.1}
 2    {'efficient_drivers': ['d1', 'd2', 'd3']}
 3                  {'fleet_average_soh': 98.5}
 4    {'frontal_wind_speed_correlation': 0.057}
 Name: common_answers, dtype: object)

In [34]:
from openai import OpenAI


# 1) Upload the JSON and create a vector store
client = OpenAI()  # requires OPENAI_API_KEY

upload_report = client.files.create(
    file=open("report.json", "rb"),
    purpose="user_data",
)

upload_csv = client.files.create(
    file=open("../curated/DEVRT-DACIA-SPRING.csv", "rb"),
    purpose="user_data",
)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/files "HTTP/1.1 200 OK"


In [35]:
upload_report, upload_csv

(FileObject(id='file-MUUbFY99D7otoy1VxvfcUR', bytes=69264, created_at=1754725273, filename='report.json', object='file', purpose='user_data', status='processed', expires_at=None, status_details=None),
 FileObject(id='file-UEosua5u86rkKhVexb1uLX', bytes=3006479, created_at=1754725274, filename='DEVRT-DACIA-SPRING.csv', object='file', purpose='user_data', status='processed', expires_at=None, status_details=None))

In [60]:
system_prompt = (
"You are an automotive data analysis expert with access to a code execution environment. "
"You will receive two files: "
"1) A CSV file containing automotive telemetry data with sensor readings, timestamps, and vehicle parameters "
"2) A JSON report containing comprehensive data quality assessment including missing values, outliers, signal validation, and statistical summaries. "
"Use the code interpreter to read, analyze, and visualize this data. "
"When answering questions: "
"- Always examine both files programmatically to understand the data structure "
"- Reference specific findings from the quality report when relevant "
"- Create visualizations and statistical analyses as needed "
"- Provide automotive domain-specific insights about signal patterns, anomalies, and data integrity "
"- If you need to perform calculations or data transformations, write and execute code "
"- Be precise and cite specific data points from your analysis "
"Answer accurately based on the data; if something is unknown or unclear, say so explicitly."
)

In [88]:
response = client.responses.create(
    model="gpt-5",
    tools=[{
      "type": "code_interpreter",
      "container": {"type": "auto",
                    "file_ids": [upload_csv.id, upload_report.id]}
    }],
    input=[
        {   
            "role": "developer",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": df_questions['question'][7],
                },
            ]
        }
    ]
)

print(response.output_text)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


309.75


In [86]:
from openai.types.responses import ResponseCodeInterpreterToolCall
print(''.join([item.code for item in response.output if isinstance(item, ResponseCodeInterpreterToolCall)]))

import json, pandas as pd, numpy as np, math, os, textwrap, sys, statistics, re, warningscsv_path = "/mnt/data/file-UEosua5u86rkKhVexb1uLX-DEVRT-DACIA-SPRING.csv"
json_path = "/mnt/data/file-MUUbFY99D7otoy1VxvfcUR-report.json"

# Load files
try:
    df = pd.read_csv(csv_path)
except Exception as e:
    df = None
    csv_err = str(e)

try:
    with open(json_path, "r") as f:
        report = json.load(f)
except Exception as e:
    report = None
    json_err = str(e)

(type(df), len(df) if df is not None else None, type(report) if report is not None else None)# Inspect columns
cols = list(df.columns) if df is not None else []
colsimport pandas as pd
import numpy as np

df_work = df.copy()

# Ensure numeric types
for col in ["speedAvg", "soh"]:
    if col in df_work.columns:
        df_work[col] = pd.to_numeric(df_work[col], errors="coerce")

# Fill missing speedAvg with mean
if "speedAvg" in df_work.columns:
    mean_speed = df_work["speedAvg"].mean(skipna=True)
    df_work["speedAvg"] =

In [ ]:
import json, pandas as pd, numpy as np, math, os, textwrap, sys, statistics, re, warningscsv_path = "/mnt/data/file-UEosua5u86rkKhVexb1uLX-DEVRT-DACIA-SPRING.csv"
json_path = "/mnt/data/file-MUUbFY99D7otoy1VxvfcUR-report.json"

# Load files
try:
    df = pd.read_csv(csv_path)
except Exception as e:
    df = None
    csv_err = str(e)

try:
    with open(json_path, "r") as f:
        report = json.load(f)
except Exception as e:
    report = None
    json_err = str(e)

(type(df), len(df) if df is not None else None, type(report) if report is not None else None)# Inspect columns
cols = list(df.columns) if df is not None else []
colsimport pandas as pd
import numpy as np

df_work = df.copy()

# Ensure numeric types
for col in ["speedAvg", "soh"]:
    if col in df_work.columns:
        df_work[col] = pd.to_numeric(df_work[col], errors="coerce")

# Fill missing speedAvg with mean
if "speedAvg" in df_work.columns:
    mean_speed = df_work["speedAvg"].mean(skipna=True)
    df_work["speedAvg"] = df_work["speedAvg"].fillna(mean_speed)

# Define bins and labels
bins = [0, 50, 80, 100, np.inf]
labels = ["0-50 km/h", "50-80 km/h", "80-100 km/h", "100+ km/h"]

# Assign bins
# We consider only non-negative speeds; negative speeds will be clipped to 0 for binning
if "speedAvg" in df_work.columns:
    speeds = df_work["speedAvg"].clip(lower=0)
    df_work["speed_bin"] = pd.cut(speeds, bins=bins, labels=labels, right=False, include_lowest=True)

# Compute average SOH per bin
if "soh" in df_work.columns and "speed_bin" in df_work.columns:
    soh_by_bin = df_work.groupby("speed_bin")["soh"].mean()
    # Round to 2 decimals
    soh_by_bin_rounded = soh_by_bin.round(2)
    soh_by_bin, soh_by_bin_rounded, mean_speed, df_work["speed_bin"].isna().sum()
else:
    soh_by_bin = None# Let's compute and return the series to view
soh_by_bin_dict = None
soh_by_bin_rounded_dict = None
mean_speed_val = None
n_na_speed_bin = None

if "soh" in df_work.columns and "speed_bin" in df_work.columns:
    soh_by_bin = df_work.groupby("speed_bin")["soh"].mean()
    soh_by_bin_rounded = soh_by_bin.round(2)
    soh_by_bin_dict = soh_by_bin.to_dict()
    soh_by_bin_rounded_dict = soh_by_bin_rounded.to_dict()
    mean_speed_val = float(df_work["speedAvg"].mean())
    n_na_speed_bin = int(df_work["speed_bin"].isna().sum())

soh_by_bin_dict, soh_by_bin_rounded_dict, mean_speed_val, n_na_speed_bin# Select bin with highest SOH based on rounded means
if soh_by_bin_dict is not None:
    soh_rounded = pd.Series(soh_by_bin_rounded_dict)
    # Ensure ordering by our labels
    soh_rounded = soh_rounded.reindex(labels)
    optimal_bin = soh_rounded.idxmax() if not soh_rounded.dropna().empty else None
else:
    optimal_bin = None

optimal_bin

In [ ]:
from openai import OpenAI
client = OpenAI()

deleted_csv = client.files.delete(upload_csv.id)

In [ ]:
deleted_report = client.files.delete(upload_report.id)

# Automotive Data Quality API Demo

This notebook demonstrates how to use the new Automotive Data Quality API to query specific column information from quality reports.

The API allows you to:
- Load JSON reports with or without the original CSV data
- Get comprehensive information about specific columns
- Query correlations, signal types, and quality issues
- Search and filter columns by various criteria

In [89]:
# Import the new API
from data_analysis_agent.automotive_data_quality_api import AutomotiveDataQualityAPI, load_quality_report

# Initialize the API with the existing report and CSV data
api = AutomotiveDataQualityAPI(
    report_path='report.json',
    csv_path='../curated/DEVRT-DACIA-SPRING.csv'
)

print(f"API initialized successfully!")
print(f"Overall quality score: {api.get_overall_quality_score():.3f}")
print(f"Total columns analyzed: {len(api.list_all_columns())}")

API initialized successfully!
Overall quality score: 0.846
Total columns analyzed: 39


In [90]:
# Get comprehensive information about a specific column
column_name = 'speedAvg'
column_info = api.get_column_info(column_name)

print(f"=== Comprehensive Information for '{column_name}' ===")
print(f"Data type: {column_info['data_type']}")
print(f"Missing percentage: {column_info['missing_percentage']}%")
print(f"Unique values: {column_info['unique_count']}")

# Signal dictionary information
signal_dict = column_info['signal_dictionary']
print(f"\nSignal Dictionary:")
print(f"  Expanded name: {signal_dict.get('expanded', 'N/A')}")
print(f"  Domain: {signal_dict.get('domain', 'N/A')}")

# Range validation
range_val = column_info.get('range_validation', {})
if range_val.get('signal_type'):
    print(f"\nRange Validation:")
    print(f"  Detected signal type: {range_val['signal_type']}")
    print(f"  Has violations: {range_val.get('has_violations', False)}")

# Quality assessment
quality = column_info.get('quality_assessment', {})
print(f"\nQuality Assessment:")
print(f"  Overall score: {quality.get('overall_score', 0):.3f}/1.0")
print(f"  Issues: {quality.get('issues', [])}")
print(f"  Strengths: {quality.get('strengths', [])}")

# Priority issues related to this column
priority_issues = column_info.get('priority_issues', [])
if priority_issues:
    print(f"\nPriority Issues:")
    for issue in priority_issues:
        print(f"  ⚠️ {issue}")

=== Comprehensive Information for 'speedAvg' ===
Data type: float64
Missing percentage: 1.29%
Unique values: 78

Signal Dictionary:
  Expanded name: speedAvg
  Domain: Unknown

Range Validation:
  Detected signal type: SPEED
  Has violations: False

Quality Assessment:
  Overall score: 1.000/1.0
  Issues: []
  Strengths: ['Low missing value rate: 1.29%']


In [91]:
# Get a human-readable summary for the column
print("=== Human-Readable Column Summary ===")
summary = api.get_column_summary('speedAvg')
print(summary)

=== Human-Readable Column Summary ===
Column: speedAvg
Domain: Unknown
Data type: float64
Missing values: 1.29%
Unique values: 78
Detected as: SPEED
Quality score: 1.00/1.0


In [92]:
# Find problematic columns based on different criteria
print("=== Problematic Columns Analysis ===")

problems = api.get_problematic_columns(
    min_missing_pct=50.0,  # Columns with >50% missing values
    min_violation_pct=1.0   # Columns with >1% range violations
)

for issue_type, columns in problems.items():
    if columns:
        print(f"\n{issue_type.replace('_', ' ').title()}: {len(columns)} columns")
        for col in columns[:5]:  # Show first 5
            print(f"  • {col}")
        if len(columns) > 5:
            print(f"  ... and {len(columns) - 5} more")

# Get overall priority issues
print(f"\n=== Overall Priority Issues ===")
priority_issues = api.get_priority_issues()
for issue in priority_issues[:5]:  # Show first 5
    print(f"⚠️ {issue}")

if len(priority_issues) > 5:
    print(f"... and {len(priority_issues) - 5} more issues")

=== Problematic Columns Analysis ===

High Missing: 3 columns
  • cars_by_speed_interval_0_80
  • cars_by_speed_interval_80_100
  • cars_by_speed_interval_100_120

Range Violations: 6 columns
  • cars_by_speed_interval_80_100
  • cars_by_speed_interval_100_120
  • cars_by_speed_interval_0_50
  • cars_by_speed_interval_50_80
  • cars_by_speed_interval_80_120
  ... and 1 more

Constant Values: 3 columns
  • soh
  • capacity
  • ref_consumption

Saturated: 5 columns
  • car_id
  • wind_mph
  • wind_kph
  • cars_by_speed_interval_0_80
  • cars_by_speed_interval_120_inf

=== Overall Priority Issues ===
⚠️ Column 'cars_by_speed_interval_0_80': 70.2% missing values
⚠️ Column 'cars_by_speed_interval_80_100': 67.8% missing values
⚠️ Column 'cars_by_speed_interval_80_100': 4.6% values outside physically possible range
⚠️ Column 'cars_by_speed_interval_100_120': 65.6% missing values
⚠️ Column 'cars_by_speed_interval_100_120': 16.4% values outside physically possible range
... and 3 more issues


In [93]:
# Analyze correlations
print("=== Correlation Analysis ===")

# Get all unexpected correlations
unexpected_corrs = api.get_correlations(correlation_type='unexpected')
print(f"Unexpected correlations found: {len(unexpected_corrs)}")

for corr in unexpected_corrs[:3]:  # Show first 3
    print(f"  {corr['column1']} ↔ {corr['column2']}: {corr['correlation']:.3f}")

# Get correlations for a specific column
speed_corrs = api.get_correlations(column_name='speedAvg')
print(f"\nCorrelations involving 'speedAvg': {len(speed_corrs)}")

# Search for columns
print(f"\n=== Column Search ===")
speed_columns = api.search_columns('speed')
print(f"Columns containing 'speed': {speed_columns}")

wind_columns = api.search_columns('wind')
print(f"Columns containing 'wind': {wind_columns}")

# Get columns by signal type (if any are detected)
latitude_cols = api.get_columns_by_signal_type('LATITUDE')
longitude_cols = api.get_columns_by_signal_type('LONGITUDE')
print(f"\nLatitude columns: {latitude_cols}")
print(f"Longitude columns: {longitude_cols}")

=== Correlation Analysis ===
Unexpected correlations found: 1
  wind_mph ↔ wind_kph: 1.000

Correlations involving 'speedAvg': 0

=== Column Search ===
Columns containing 'speed': ['speedAvg', 'cars_by_speed_interval_0_80', 'cars_by_speed_interval_80_100', 'cars_by_speed_interval_100_120', 'cars_by_speed_interval_0_50', 'cars_by_speed_interval_50_80', 'cars_by_speed_interval_80_120', 'cars_by_speed_interval_120_inf', 'max_speed']
Columns containing 'wind': ['wind_mph', 'wind_kph', 'wind_degree', 'wind_dir', 'Frontal_Wind']

Latitude columns: ['latitude']
Longitude columns: ['longitude']


In [94]:
# Demonstrate convenience functions
from data_analysis_agent.automotive_data_quality_api import get_column_quality_summary

print("=== Convenience Functions ===")

# Quick column summary without creating API instance
quick_summary = get_column_quality_summary('report.json', 'soc')
print("Quick summary for 'soc' column:")
print(quick_summary)

# Show basic dataset statistics
print(f"\n=== Dataset Statistics ===")
basic_stats = api.get_basic_stats()
print(f"Shape: {basic_stats['shape']}")
print(f"Memory usage: {basic_stats['memory_usage_mb']} MB")
print(f"Missing percentage: {basic_stats['missing_percentage']}%")
print(f"Numeric columns: {basic_stats['numeric_columns']}")
print(f"Non-numeric columns: {basic_stats['non_numeric_columns']}")

# List all available columns
print(f"\n=== All Available Columns ===")
all_columns = api.list_all_columns()
print(f"Total columns: {len(all_columns)}")
print("First 10 columns:", all_columns[:10])

=== Convenience Functions ===
Quick summary for 'soc' column:
Column: soc
Domain: Unknown
Data type: int64
Missing values: 0.0%
Unique values: 60
Quality score: 1.00/1.0

=== Dataset Statistics ===
Shape: [8423, 39]
Memory usage: 7.58 MB
Missing percentage: 8.19%
Numeric columns: 28
Non-numeric columns: 11

=== All Available Columns ===
Total columns: 39
First 10 columns: ['soc', 'soh', 'route_id', 'longitude', 'latitude', 'altitude', 'timestamp_gps_utc', 'car_id', 'time_diff', 'point_geom']
